# Compare accuracy of ResNet18 CNN with BCE loss function and CNN with advanced BCE loss function on Kaggle Cat-Dog dataset

In [ ]:
import time
import copy
import random
import os
import glob

import torch
import torchvision
import numpy as np
import cv2
from PIL import Image

from tqdm import tqdm
import matplotlib.pyplot as plt

# Parameters

In [ ]:
EPOCHS = 3
BATCH_SIZE = 16
LR = 0.001
LABELS = ['Cat', 'Dog']

if torch.cuda.is_available():
    TARGET = 'cuda'
else:
    TARGET = 'cpu'

# Classes for load Cat-Dog dataset

In [ ]:
class CatDog():
    def __init__(self, path):
        self.img_size = 100
    
        self.train_data = []
        self.cat_path = os.path.normpath(path) + '/Cat'
        self.dog_path = os.path.normpath(path) + '/Dog'
        
        self.labels = {self.cat_path: 0, self.dog_path: 1}
    
    def get_train_data(self):
        for label in self.labels:
            images_path = glob.glob(label + '/*.jpg')
            images_path.sort()
            
            for image_path in tqdm(images_path):
                try:
                    img = cv2.imread(image_path)
                    img = cv2.resize(img, (self.img_size, self.img_size))
                    
                    img = Image.fromarray(img)
                    img = torchvision.transforms.ToTensor()(img)
                    
                    ans = np.eye(2)[self.labels[label]]
                    ans = torch.tensor(ans, dtype=torch.float32)
                    
                    self.train_data.append([img, ans])
                except:
                    pass
        
        return self.train_data

class CatDogDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.__data = data
        
    def __getitem__(self, index):
        return (self.__data[index][0], self.__data[index][1])
    
    def __len__(self):
        return len(self.__data)

# Load dataset and split to "train" and "test" datasets

In [ ]:
cat_dog = CatDog(path='PetImages/')
cat_dog_dataset = cat_dog.get_train_data()

np.random.shuffle(cat_dog_dataset)
print('Dataset length', len(cat_dog_dataset))

cat_dog_train = cat_dog_dataset[:int(len(cat_dog_dataset) * 0.8)]
cat_dog_test = cat_dog_dataset[int(len(cat_dog_dataset) * 0.8):]

print('Train length', len(cat_dog_train))
print('Test length', len(cat_dog_test))

train = CatDogDataset(cat_dog_train)
test = CatDogDataset(cat_dog_test)

train_dataloader = torch.utils.data.DataLoader(train, batch_size=BATCH_SIZE, num_workers=4, shuffle=True)

# CNN train

In [ ]:
net = torchvision.models.resnet18(pretrained=False, num_classes=2).to(TARGET)
optimizer = torch.optim.Adam(net.parameters(), lr=LR)
mse_loss = torch.nn.MSELoss()

best_net = torchvision.models.resnet18(pretrained=False, num_classes=2).to(TARGET)
best_loss = torch.tensor(10e10)

for epoch in range(EPOCHS):
    time.sleep(1)
    for data in tqdm(train_dataloader): 
        x_train, y_train = data
        x_train /= 255.0
        
        x_train = x_train.to(TARGET)
        y_train = y_train.to(TARGET)
    
#         net.zero_grad() 
        optimizer.zero_grad() 
        output = net(x_train)
        
        loss = mse_loss(output, y_train)
        
        loss.backward()
        optimizer.step()
    
    print('Loss', loss.item())
    
    if loss < best_loss:
        best_net = copy.deepcopy(net)
        best_loss = loss

time.sleep(0.1)
print('Best loss', best_loss.item())

In [ ]:
i = random.randint(0, len(test) - 1)
print('Item', i)

net_result = best_net(test[i][0].unsqueeze(0).to(TARGET))
real = test[i][1]

p = np.argmax(net_result.cpu().detach().numpy())
r = np.argmax(real.detach().numpy())

print('Predict:', LABELS[p])
print('Real:', LABELS[r])

img = np.array(test[i][0].permute(1, 2, 0))
img = img[:, :, ::-1].copy()
plt.imshow(img)
plt.show()

# CNN train with advanced loss calculation

In [ ]:
net_adv = torchvision.models.resnet18(pretrained=False, num_classes=2).to(TARGET)
optimizer_adv = torch.optim.Adam(net_adv.parameters(), lr=LR)
mse_loss_adv = torch.nn.MSELoss()

best_net_adv = torchvision.models.resnet18(pretrained=False, num_classes=2).to(TARGET)
best_loss_adv = torch.tensor(10e10)

for epoch in range(EPOCHS):
    time.sleep(1)
    for data in tqdm(train_dataloader): 
        x_train, y_train = data
        x_train /= 255.0
        
        x_train = x_train.to(TARGET)
        y_train = y_train.to(TARGET)
    
        optimizer_adv.zero_grad() 
#         net_adv.zero_grad() 
        output = net_adv(x_train)
    
        obj = torch.zeros(output.size()).bool().to(TARGET)
        no_obj = torch.zeros(output.size()).fill_(1).bool().to(TARGET)
        
        for i in range(y_train.size()[0]):
            pos = np.argmax(y_train[i].cpu().detach().numpy())
            obj[i, pos] = 1
            
        for i in range(y_train.size()[0]):
            pos = np.argmax(y_train[i].cpu().detach().numpy())
            no_obj[i, pos] = 0     
        
        loss_obj = mse_loss_adv(output[obj], y_train[obj])
        loss_no_obj = mse_loss_adv(output[no_obj], y_train[no_obj])
        loss_adv = 1.0 * loss_obj + 2.0 * loss_no_obj
        
        loss_adv.backward()
        optimizer_adv.step()
    
    print('Loss', loss_adv.item())
    
    if loss_adv < best_loss_adv:
        best_net_adv = copy.deepcopy(net_adv)
        best_loss_adv = loss_adv

time.sleep(0.1)
print('Best loss', best_loss_adv.item())

In [ ]:
i = random.randint(0, len(test) - 1)
print('Item', i)

net_result = best_net_adv(test[i][0].unsqueeze(0).to(TARGET))
real = test[i][1]

p = np.argmax(net_result.cpu().detach().numpy())
r = np.argmax(real.detach().numpy())

print('Predict:', LABELS[p])
print('Real:', LABELS[r])

img = np.array(test[i][0].permute(1, 2, 0))
img = img[:, :, ::-1].copy()
plt.imshow(img)
plt.show()

# Results

In [ ]:
i = random.randint(0, len(test) - 1)
print('Item', i)

net_result = best_net(test[i][0].unsqueeze(0).cuda())
adv_net_result = best_net_adv(test[i][0].unsqueeze(0).cuda())
real = test[i][1]

p = np.argmax(net_result.cpu().detach().numpy())
p_adv = np.argmax(adv_net_result.cpu().detach().numpy())
r = np.argmax(real.detach().numpy())

print('Predict', LABELS[p])
print('Predict (adv loss)', LABELS[p_adv])
print('Real', LABELS[r])

img = np.array(test[i][0].permute(1, 2, 0))
img = img[:, :, ::-1].copy()
plt.imshow(img)
plt.show()

# Number of errors for every network

In [ ]:
net_wr = 0
adv_net_wr = 0

for t_data in tqdm(test):
    net_result = best_net(t_data[0].unsqueeze(0).to(TARGET))
    adv_net_result = best_net_adv(t_data[0].unsqueeze(0).to(TARGET))
    
    net_result = np.argmax(net_result.cpu().detach().numpy())
    adv_net_result = np.argmax(adv_net_result.cpu().detach().numpy())
    
    real = np.argmax(t_data[1].cpu().detach().numpy())
    
    if net_result != real:
        net_wr += 1
    
    if adv_net_result != real:
        adv_net_wr += 1

time.sleep(0.1)
print('Predict errors', net_wr)
print('Predict errors (adv loss)', adv_net_wr)

print('Accuracy', (1 - float(net_wr / len(test))) * 100)
print('Accuracy (adv loss)', (1 - float(adv_net_wr / len(test))) * 100)

# Save models

In [ ]:
torch.save(best_net.state_dict(), 'resnet18.pt')
torch.save(best_net_adv.state_dict(), 'resnet18_adv_loss.pt')